<a href="https://colab.research.google.com/github/moajjem04/Pytorch_Practice/blob/main/Robot_Data_Processing_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Accessing Google Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%cd '/content/drive/MyDrive/robot'

/content/drive/.shortcut-targets-by-id/1xoj4fkjFWbgZZGpDNuTyZsH1-eOzncVi/robotic arm data - s


In [4]:
%pwd

'/content/drive/.shortcut-targets-by-id/1xoj4fkjFWbgZZGpDNuTyZsH1-eOzncVi/robotic arm data - s'

In [5]:
%ls

data_v2.pkl                  Samling_for_Zhen_Lizhuoxun_S.csv
IMG_4598.MOV                 Samling_for_Zhen_l_s.csv
IMG_4599.MOV                 Samling_for_Zhen_lyfg_2e_s.csv
IMG_4600.MOV                 Samling_for_Zhen_lygf_e_s.csv
IMG_4601.MOV                 Samling_for_Zhen_oyg_2e_s.csv
IMG_4602.MOV                 Samling_for_Zhen_oyg_e_s1.csv
Samling_for_Zhen_d_s2.csv    Samling_for_Zhen_w_s.csv
Samling_for_Zhen_d_s.csv     Samling_for_Zhen_x_s.csv
Samling_for_Zhen_g_S.csv     Samling_for_Zhen_yg_e_s.csv
Samling_for_Zhen_lg_e_s.csv


# Loading the Data

In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from google.colab import files
import os
import pickle

In [7]:
file_name = "data_v2.pkl"
open_file = open(file_name, "rb")
Data = pickle.load(open_file)
open_file.close()

In [8]:
Data.keys()

dict_keys(['Zhen_d_s', 'Zhen_d_s2', 'Zhen_g_S', 'Zhen_l_s', 'Zhen_lg_e_s', 'Zhen_Lizhuoxun_S', 'Zhen_lyfg_2e_s', 'Zhen_lygf_e_s', 'Zhen_oyg_2e_s', 'Zhen_oyg_e_s1', 'Zhen_w_s', 'Zhen_x_s', 'Zhen_yg_e_s'])

In [9]:
count = 0
for key in Data.keys():
  _,a = Data[key]
  count+= a.shape[0]
  print(key,a.shape)
print('Total Samples:',count)

Zhen_d_s (189, 21, 5000, 1)
Zhen_d_s2 (189, 21, 5000, 1)
Zhen_g_S (231, 21, 5000, 1)
Zhen_l_s (237, 21, 5000, 1)
Zhen_lg_e_s (237, 21, 5000, 1)
Zhen_Lizhuoxun_S (242, 21, 5000, 1)
Zhen_lyfg_2e_s (242, 21, 5000, 1)
Zhen_lygf_e_s (242, 21, 5000, 1)
Zhen_oyg_2e_s (242, 21, 5000, 1)
Zhen_oyg_e_s1 (242, 21, 5000, 1)
Zhen_w_s (269, 21, 5000, 1)
Zhen_x_s (269, 21, 5000, 1)
Zhen_yg_e_s (269, 21, 5000, 1)
Total Samples: 3100


In [10]:
count/(5*13)

47.69230769230769

Lets take 45 samples from each class

In [11]:
#count = 0
for key in Data.keys():
  _,a = Data[key]
  #count+= a.shape[0]
  b = a.shape[0]
  b = round(45*100/b)
  print(key,b)
#print('Total Samples:',count)

Zhen_d_s 24
Zhen_d_s2 24
Zhen_g_S 19
Zhen_l_s 19
Zhen_lg_e_s 19
Zhen_Lizhuoxun_S 19
Zhen_lyfg_2e_s 19
Zhen_lygf_e_s 19
Zhen_oyg_2e_s 19
Zhen_oyg_e_s1 19
Zhen_w_s 17
Zhen_x_s 17
Zhen_yg_e_s 17


## Defining some helper functions

In [12]:
def restructure_time_data(time_stamp):
  '''
  The previous time stamps are arranged in a way that all the points of time are listed.
  However, they can be restructered so that the dictionary consists of a list that has
  the start and ending of the signal segment.
  '''
  start_end_time ={}
  for key in time_stamp.keys():
    temp1 = time_stamp[key]
    time = [temp1.iloc[0],temp1.iloc[-1]]
    start_end_time[key]= time
  
  return start_end_time

In [13]:
def split_time_data(time_stamp,val_size):
  '''
  Splitting the time_data from start till we have only a specific
  amount of signals left.
  Input:
    time_stamp - a dictionary containing the time data
    val_size   - integer that specifies the amount of validation signals
  '''
  idx = len(time_stamp)- val_size
  train_time = {}
  val_time   = {}
  for key in time_stamp.keys():
    if key < idx:
      train_time[key] = time_stamp[key]
    else :
      val_time[key]   = time_stamp[key]
  return train_time, val_time


# Preparing the Data

In [14]:
name_class = {}
count = 0
for key in Data.keys():
  name_class[key] = count
  count += 1

`name_class` is going to be used to change the names of the subjects to integers. We will later use One-hot encoding to train our model.

In [15]:
for key in name_class:
  print(key,':',name_class[key])

Zhen_d_s : 0
Zhen_d_s2 : 1
Zhen_g_S : 2
Zhen_l_s : 3
Zhen_lg_e_s : 4
Zhen_Lizhuoxun_S : 5
Zhen_lyfg_2e_s : 6
Zhen_lygf_e_s : 7
Zhen_oyg_2e_s : 8
Zhen_oyg_e_s1 : 9
Zhen_w_s : 10
Zhen_x_s : 11
Zhen_yg_e_s : 12


Main Script

In [16]:
time_dict = {}
first_flag = True
# Main Loop
for class_key in Data.keys():
  time_stamp,sig = Data[class_key]               # Unpacking the data of each class
  time_stamp = restructure_time_data(time_stamp)  # restructuring the time data

  # Split into train and validation set
  val_size = 45 # as predetermined

  train_time, val_time = split_time_data(time_stamp, val_size)  # split the time stamp
  temp_train_sig = sig[:-val_size]                              # split the signal
  temp_val_sig   = sig[-val_size:]                              # ...

  temp_train_class = np.ones((temp_train_sig.shape[0],1))       # creating empty np array for class data
  temp_val_class   = np.ones((val_size,1))

  class_value = name_class[class_key]                           # getting the class value from the dictionary
  temp_train_class.fill(class_value)                            # assigning class data
  temp_val_class.fill(class_value)
  # Saving the data
  time_dict[class_key] = (train_time,val_time)                 # saving time data

  if first_flag:                                                # for first time
    train_sig   = temp_train_sig
    val_sig     = temp_val_sig
    train_class = temp_train_class
    val_class   = temp_val_class
    first_flag = False
  else:
    train_sig   = np.concatenate((train_sig,  temp_train_sig))
    val_sig     = np.concatenate((val_sig,    temp_val_sig))
    train_class = np.concatenate((train_class,temp_train_class))
    val_class   = np.concatenate((val_class,  temp_val_class))
  

In [17]:
print('The total data')
print('Training Data shape:',train_sig.shape)
print('Validation Data shape:',val_sig.shape)
print('Training Class shape:',train_class.shape)
print('Validation Class shape:',val_class.shape)
temp = val_sig.shape[0]/(train_sig.shape[0] + val_sig.shape[0])
print('Proportion of Validation Data:',round(temp*100,2),'%')

The total data
Training Data shape: (2515, 21, 5000, 1)
Validation Data shape: (585, 21, 5000, 1)
Training Class shape: (2515, 1)
Validation Class shape: (585, 1)
Proportion of Validation Data: 18.87 %


## Shuffling the Data

In [18]:
np.random.seed(42)

In [19]:
train_idx = np.arange(0,train_sig.shape[0])
val_idx = np.arange(0,val_sig.shape[0])

In [20]:
np.random.shuffle(train_idx)
np.random.shuffle(val_idx)

In [21]:
train_idx

array([ 617,  927,  942, ..., 1130, 1294,  860])

In [22]:
train_sig   = train_sig[train_idx,:,:,:]
train_class = train_class[train_idx,:]
val_sig     = val_sig[val_idx,:,:,:]
val_class   = val_class[val_idx,:]

In [23]:
train_class[0]

array([3.])

# Defining the Neural Network Architecture

In [24]:
import torch
import torch.nn as nn
import torch.nn.functional as F
# import torch.optim as optim
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv1d, MaxPool1d, Module, Softmax, BatchNorm1d, Dropout, MSELoss
from torch.optim import Adam, SGD

from tqdm import tqdm

In [25]:
class Net(nn.Module):
  def __init__(self,n_channels):
    super().__init__()

    self.cnn_layers = Sequential(
        # Defining a 1D convolution layer
        Conv1d(n_channels, 32, kernel_size=5, padding=1),
        BatchNorm1d(32),
        ReLU(inplace=True),
        MaxPool1d(kernel_size=2, stride=2),
        # Defining another 1D convolution layer
        Conv1d(32, 64, kernel_size=5, padding=1),
        BatchNorm1d(64),
        ReLU(inplace=True),
        MaxPool1d(kernel_size=2, stride=2),
        # Defining another 1D convolution layer
        Conv1d(64, 64, kernel_size=5, padding=1),
        BatchNorm1d(64),
        ReLU(inplace=True),
        MaxPool1d(kernel_size=2, stride=2),
    )
    # This code is used calculate the size of the flattened layer by having one
    #   pass forward
    x = torch.randn(21,5000).view(-1,21,5000)
    self._to_linear = None
    self.convs(x)

    self.flat = nn.Flatten()

    self.linear_layers = Sequential(
                Linear(self._to_linear, 13)
            )

  def convs(self, x):
    # max pooling over 2x2
    x = self.cnn_layers(x)
    #print(x[0].shape)

    if self._to_linear is None:
      self._to_linear = x[0].shape[0]*x[0].shape[1]
    return x

  def forward(self, x):
    x = self.convs(x)
    x = self.flat(x)
    x = self.linear_layers(x)
    return x

In [26]:
net = Net(21)
print(net)

Net(
  (cnn_layers): Sequential(
    (0): Conv1d(21, 32, kernel_size=(5,), stride=(1,), padding=(1,))
    (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv1d(32, 64, kernel_size=(5,), stride=(1,), padding=(1,))
    (5): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv1d(64, 64, kernel_size=(5,), stride=(1,), padding=(1,))
    (9): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace=True)
    (11): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (flat): Flatten(start_dim=1, end_dim=-1)
  (linear_layers): Sequential(
    (0): Linear(in_features=39872, out_features=13, bias=True)
  )
)


In [27]:
from torchsummary import summary
summary(net.cuda(), (21,5000))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1             [-1, 32, 4998]           3,392
       BatchNorm1d-2             [-1, 32, 4998]              64
              ReLU-3             [-1, 32, 4998]               0
         MaxPool1d-4             [-1, 32, 2499]               0
            Conv1d-5             [-1, 64, 2497]          10,304
       BatchNorm1d-6             [-1, 64, 2497]             128
              ReLU-7             [-1, 64, 2497]               0
         MaxPool1d-8             [-1, 64, 1248]               0
            Conv1d-9             [-1, 64, 1246]          20,544
      BatchNorm1d-10             [-1, 64, 1246]             128
             ReLU-11             [-1, 64, 1246]               0
        MaxPool1d-12              [-1, 64, 623]               0
          Flatten-13                [-1, 39872]               0
           Linear-14                   

In [28]:
torch.cuda.is_available()

True

In [29]:
optimizer = Adam(net.parameters(),lr = 1e-3)
#criterion = CrossEntropyLoss()
criterion = MSELoss()
if torch.cuda.is_available():
  net = net.cuda()
  criterion = criterion.cuda()

In [33]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
a = enc.fit_transform(train_y)

TypeError: ignored

In [ ]:
# converting training images into torch format
train_x = train_sig.astype(float)
train_x = train_x.reshape(train_sig.shape[0], 21,5000)
train_x  = torch.from_numpy(train_x)

# converting the target into torch format
train_y = train_class.astype(int)
#train_y = enc.fit_transform(train_y)
train_y = torch.from_numpy(train_y)

# shape of training data
train_x.shape, train_y.shape

In [ ]:
type(train_x[0])

In [ ]:
# converting training images into torch format
val_x = val_sig.astype(float)
val_x = val_sig.reshape(val_sig.shape[0], 21,5000)
val_x  = torch.from_numpy(val_x)

# converting the target into torch format
val_y = val_class.astype(int)
val_y = 
val_y = torch.from_numpy(val_y)

# shape of training data
val_x.shape, val_y.shape

In [ ]:
print(a[0])

In [ ]:
def train(model,batch_size,epochs):
    tr_loss = 0
    for epoch in range(epochs):
      for i in tqdm(range(0,len(train_y),batch_size)):
        batch_X = train_x[i:i+batch_size].float()
        batch_y = train_y[i:i+batch_size]

        if torch.cuda.is_available():
          batch_X = batch_X.cuda()
          batch_y = batch_y.cuda()

        model.zero_grad()
        output_train = model(batch_X)
        print(output_train.shape)
        print(output_train)
        print(batch_y.shape)
        loss_train = criterion(output_train, batch_y)
        train_losses.append(loss_train)
        # computing the updated weights of all the model parameters
        loss_train.backward()
        optimizer.step()
        tr_loss = loss_train.item()
      with torch.no_grad():
        for j in tqdm(range(0,len(val_y),batch_size)):
          val_batch_X = val_x[i:i+batch_size].float()
          val_batch_y = val_y[i:i+batch_size]

          if torch.cuda.is_available():
            val_batch_X = val_batch_X.cuda()
            val_batch_y = val_batch_y.cuda()

          output_val = model(val_batch_X)
          loss_val = criterion(output_val, val_batch_y)
          val_losses.append(loss_val)   
      if (epoch+1)%10 == 0:
        # printing the validation loss
          print('Epoch : ',epoch+1, '\t', 'loss :', loss_val)     


In [ ]:
val_y.shape

In [ ]:
# defining the number of epochs
n_epochs = 200
batch_size = 32
# empty list to store training losses
train_losses = []
# empty list to store validation losses
val_losses = []
# training the model
train(net,batch_size,n_epochs)